In [1]:
!pip install transformers torch accelerate bitsandbytes pandas tqdm sentencepiece --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 21.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not

In [2]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from kaggle_secrets import UserSecretsClient
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
import warnings

In [3]:
# === 2. HIDE THE TOKENIZER WARNING ===
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
print("Tokenizer parallelism warning suppressed.")

# === 3. CONFIGURE AND LOAD THE MODEL ===
# This is the new model ID for StableLM Zephyr (3B version)
model_id = "stabilityai/stablelm-zephyr-3b"
model = None
tokenizer = None

try:
    # Config for 4-bit quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    # This model requires 'trust_remote_code=True'
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    
    print(f"Loading model: {model_id}...")
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    print("Model loaded successfully on GPU.")

except Exception as e:
    print(f"Error loading model: {e}")
    print("This might be a Kaggle GPU memory issue or a network problem. Try restarting the session.")


# === 4. DEFINE THE PERPLEXITY FUNCTION ===
def get_perplexity(text, model, tokenizer):
    if not text or not isinstance(text, str):
        return float('inf')
    try:
        inputs = tokenizer(text, return_tensors="pt").to(model.device)
        input_ids = inputs.input_ids
        with torch.no_grad():
            outputs = model(input_ids, labels=input_ids)
        mean_nll = outputs.loss
        perplexity = torch.exp(mean_nll)
        return perplexity.item()
    except Exception as e:
         return float('inf')

print("\n--- Setup Complete ---")

Tokenizer parallelism warning suppressed.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Loading model: stabilityai/stablelm-zephyr-3b...


config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

2025-10-24 16:15:23.406914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761322523.801238      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761322523.899689      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model loaded successfully on GPU.

--- Setup Complete ---


In [4]:
# === 5. LOAD DATA AND RUN AUDIT ===
# Check if the model from Cell 1 was loaded successfully
if 'model' in locals() and model is not None:
    
    # --- Load and Filter Dataset ---
    print("Loading CrowS-Pairs dataset using pandas...")
    # This path comes from adding the dataset via "+ Add Input"
    file_path = "/kaggle/input/a-dataset-for-measuring-social-biases-in-mlms/crows_pairs_anonymized.csv"
    
    try:
        df = pd.read_csv(file_path)
        print(f"✅ Loaded {len(df)} records from CSV.")
        
        if 'bias_type' not in df.columns:
            print("Error: Column 'bias_type' not found in CSV.")
            gender_pairs_df = None
        else:
            gender_pairs_df = df[df['bias_type'] == 'gender'].copy()
            print(f"✅ Filtered {len(gender_pairs_df)} 'gender' pairs.")
            if len(gender_pairs_df) == 0:
                print("Stopping execution: No gender pairs found after filtering.")
                gender_pairs_df = None
                
    except FileNotFoundError:
        print(f"Error: CSV file not found at {file_path}")
        print("Please use the '+ Add Input' button in the Kaggle sidebar to add the dataset.")
        gender_pairs_df = None
    except Exception as e:
        print(f"An error occurred loading or filtering data: {e}")
        gender_pairs_df = None

    # --- Run Audit (Only if data loading was successful) ---
    if gender_pairs_df is not None:
        bias_score_count = 0
        processed_pairs = 0
        total_pairs_to_process = len(gender_pairs_df)

        print(f"\nRunning audit on {total_pairs_to_process} gender pairs for {model_id}...")
        start_time = time.time()

        for index, pair in tqdm(gender_pairs_df.iterrows(), total=total_pairs_to_process):
            try:
                sent_more_stereo = pair['sent_more']
                sent_less_anti_stereo = pair['sent_less']

                if not isinstance(sent_more_stereo, str) or not isinstance(sent_less_anti_stereo, str) or not sent_more_stereo or not sent_less_anti_stereo:
                    continue

                ppl_stereo = get_perplexity(sent_more_stereo, model, tokenizer)
                ppl_anti_stereo = get_perplexity(sent_less_anti_stereo, model, tokenizer)

                if ppl_stereo == float('inf') or ppl_anti_stereo == float('inf'):
                    continue

                processed_pairs += 1

                if ppl_stereo < ppl_anti_stereo:
                    bias_score_count += 1

            except Exception as e:
                print(f"Loop error processing index {index}: {e}")
                continue

        end_time = time.time()
        print("Audit complete!")
        run_duration = end_time - start_time

        # --- Calculate and Print Final Score ---
        final_bias_score = (bias_score_count / processed_pairs) * 100 if processed_pairs > 0 else 0

        print("\n" + "="*30)
        print(f"      FINAL RESULTS FOR: {model_id}")
        print("="*30)
        print(f"Total pairs attempted: {total_pairs_to_process}")
        print(f"Pairs successfully processed: {processed_pairs}")
        print(f"Pairs where stereotype was preferred: {bias_score_count}")
        print(f"Audit duration: {run_duration:.2f} seconds ({run_duration/60:.2f} minutes)")
        print(f"BIAS SCORE (Higher is worse): {final_bias_score:.2f}%")
        print("="*30)

else:
    print("Model not loaded from Cell 1. Please run Cell 1 successfully before running Cell 2.")

Loading CrowS-Pairs dataset using pandas...
✅ Loaded 1508 records from CSV.
✅ Filtered 262 'gender' pairs.

Running audit on 262 gender pairs for stabilityai/stablelm-zephyr-3b...


100%|██████████| 262/262 [02:11<00:00,  1.99it/s]

Audit complete!

      FINAL RESULTS FOR: stabilityai/stablelm-zephyr-3b
Total pairs attempted: 262
Pairs successfully processed: 262
Pairs where stereotype was preferred: 165
Audit duration: 131.63 seconds (2.19 minutes)
BIAS SCORE (Higher is worse): 62.98%
